Daftar Scrapping
1. Review
2. Seat Type
3. Seat Comfort
4. Cabin Staff Service
5. Food & Beverages
6. Value For Money
7. Recommended

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from numpy import NaN

In [ ]:
page_number = 10
pages_size = 100

seat_type = []
recommended = []
datetimes = []
text_reviews = []

# review star fill
seat_comfort_ratings = []
cabin_staff_service_ratings = []
food_beverages_ratings = []
inflight_entertainment_ratings = []
ground_service_ratings = []
wifi_connectivity_ratings = []

for i in range(1,page_number+1):
  # request ke url website
  url = f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize={pages_size}"
  response = requests.get(url)
  html = response.text

  # membuat objek BeautifulSoup
  soup = BeautifulSoup(html, "html.parser")

  # mengambil informasi seat type
  td_elements = soup.find_all("td", {"class": "review-value"})
  for td in td_elements:
    if "Seat Type" in td.previous_sibling.text:
      seat_type.append(td.text)

  # mengambil informasi recommend yes/no
  td_recommend = soup.find_all("td", {"class": "review-rating-header recommended"})
  for recommend in td_recommend:
    recommended.append(recommend.next_sibling.text)

  # mengambil tanggal tiap review
  date = soup.find_all('time', itemprop='datePublished')
  for dates in date:
    datetimes.append(dates['datetime'])

  # mengambil informasi text review
  for para in soup.find_all("div", {"class": "text_content"}):
    text_reviews.append(para.get_text())  

  # mengambil informasi review starfill
  review_containers = soup.select('article[itemprop="review"]')
  for review in review_containers:
    # seat comfort
    seat_comfort = review.select_one('td.review-rating-header.seat_comfort')
    if seat_comfort is not None:
      seat_comfort_rating = seat_comfort.find_next_sibling('td').select('span.star.fill')[-1].text
      seat_comfort_ratings.append(seat_comfort_rating)
    else:
      # handle the case where seat_comfort is None
      seat_comfort_ratings.append(NaN)

    # cabin staff service
    cabin_staff_service = review.select_one('td.review-rating-header.cabin_staff_service')
    if cabin_staff_service is not None:
      cabin_staff_service_rating = cabin_staff_service.find_next_sibling('td').select('span.star.fill')[-1].text
      cabin_staff_service_ratings.append(cabin_staff_service_rating)
    else:
      # handle the case where food_beverages is None
      cabin_staff_service_ratings.append(NaN)
    
    # food beverages
    food_beverages = review.select_one('td.review-rating-header.food_and_beverages')
    if food_beverages is not None:
      food_beverages_rating = food_beverages.find_next_sibling('td').select('span.star.fill')[-1].text
      food_beverages_ratings.append(food_beverages_rating)
    else:
      # handle the case where food_beverages is None
      food_beverages_ratings.append(NaN)

    # ground_service
    ground_service = review.select_one('td.review-rating-header.ground_service')
    if ground_service is not None:
      ground_service_rating = ground_service.find_next_sibling('td').select('span.star.fill')[-1].text
      ground_service_ratings.append(ground_service_rating)
    else:
      # handle the case where food_beverages is None
      ground_service_ratings.append(NaN)

In [ ]:
# Membuat dataframe
df = pd.DataFrame()

# Memasukkan kolom dari list yang sudah dibuat dari hasil scrapping
df['reviews'] = text_reviews
df['date_reviews'] = datetimes
df['seat_type'] = seat_type
df['seat_comfort_ratings'] = seat_comfort_ratings
df['cabin_staff_service_ratings'] = cabin_staff_service_ratings
df['food_beverages_ratings'] = food_beverages_ratings
df['ground_service'] = ground_service_ratings
df['recommended'] = recommended

In [ ]:
df.to_csv('British Airways.csv', index=False)

In [ ]:
df.head()

,reviews,date_reviews,seat_type,seat_comfort_ratings,cabin_staff_service_ratings,food_beverages_ratings,ground_service,recommended
0,✅ Trip Verified | Despite being a gold member...,2023-05-01,Economy Class,4,3,3,1,no
1,Not Verified | Regarding the aircraft and seat...,2023-04-28,Business Class,4,2,1,3,no
2,Not Verified | I travelled with British Airway...,2023-04-26,Economy Class,3,3,3,3,no
3,Not Verified | Food was lousy. Who ever is pl...,2023-04-24,Premium Economy,3,2,1,3,no
4,✅ Trip Verified | Had the worst experience. Th...,2023-04-24,Premium Economy,2,1,2,1,no


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   reviews                      1000 non-null   object
 1   date_reviews                 1000 non-null   object
 2   seat_type                    1000 non-null   object
 3   seat_comfort_ratings         924 non-null    object
 4   cabin_staff_service_ratings  917 non-null    object
 5   food_beverages_ratings       799 non-null    object
 6   ground_service               948 non-null    object
 7   recommended                  1000 non-null   object
dtypes: object(8)
memory usage: 62.6+ KB
